# Day Activities

In [19]:
import numpy as np

LIKELIHOOD = np.array([[0.25, 0.75],
                       [1, 0]
                       ])

def bayes_filter(prior: np.ndarray):
    numerator = LIKELIHOOD @ prior
    normalizer = np.sum(numerator)
    return numerator / normalizer

# Bayes loop from N = 1,2,...,10

# Initialize prior = [nf ; f] <- column vector
prior = np.array([
                [0.985],
                [0.015]
                ])

for i in range(10):
    print(f"Posterior t={i+1}:")
    pxz = bayes_filter(prior)
    print(round(pxz[1][0],3))
    # update prior
    prior = pxz

Posterior t=1:
0.793
Posterior t=2:
0.243
Posterior t=3:
0.671
Posterior t=4:
0.36
Posterior t=5:
0.598
Posterior t=6:
0.423
Posterior t=7:
0.556
Posterior t=8:
0.457
Posterior t=9:
0.532
Posterior t=10:
0.476
